## Step 11: 가변 길이 인수(순전파 편)

입출력 변수는 여러 개일 수도 있다. 이를 고려하여 DeZero가 가변 길이 입출력을 처리할 수 있도록 확장해보도록 하자.

### 11.1 Function 클래스 수정

가변 길이 입출력을 표현하기 위해서는 변수들을 리스트(또는 튜플)에 넣어 처리하면 간편하다. 즉, Function 클래스는 '하나의 인수'로 리스트를 받고 마찬가지로 '하나의 값'으로 리스트를 반환하는 것이다.

현재 구현되어 있는 Function 클래스는 다음과 같다.

```python
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(as_array(y))
        output.set_creator(self)
        self.input = input
        self.output = output
        return output
    
    def forward(self, x):
        raise NotImplementedError()

    
    def backward(self, gy):
        raise NotImplementedError()
```

Function의 __call__ 메서드는 Variable에서 데이터를 꺼낸 후 forward 메서드에서 구체적인 계산을 수행하는 형태이다. 이후 계산 결과를 Variable에 넣고 창조자 표시를 한 후 이를 반환한다.

이러한 로직을 염두에 두고 __call__ 메서드의 인수와 반환값을 리스트로 바꿔보도록 하자.

```python
class Function:
    def __call__(self, inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(xs)
        outputs = [Variable(as_array(y)) for y in ys]

        for output in outputs:
            output.set_creator(self)
        
        self.inputs = inputs
        self.outputs = outputs
        return outputs
    
    def forward(self, xs):
        raise NotImplementedError()

    
    def backward(self, gys):
        raise NotImplementedError()
```

인수와 반환값을 리스트로 변경했다. 해당 리스트의 생성에서는 for문 대신 list comprehension을 사용해 처리 속도를 높였다.

### 11.2 Add 클래스 구현

이어서 구현된 새로운 Function 클래스를 사용해 구체적인 함수 Add를 구현해보도록 하겠다.

```python
class Add(Function):
    def forward(self, xs):
        x0, x1 = xs
        y = x0 + x1
        return (y,)
```

Add 클래스는 인수로 두 개의 변수가 담긴 리스트를 받는다. 따라서 이를 unpacking을 이용해 x0과 x1에 각각 할당해주었다. 이후 덧셈 계산을 수행하여 y에 저장하였고, 이를 튜플 형태로 반환하였다.

수정된 Add 클래스는 다음과 같이 사용 가능하다.

```python
xs = [Variable(np.array(2)), Variable(np.array(3))] # 리스트 형태로 input 준비
f = Add()
ys = f(xs)
y = ys[0] # 튜플 첫번째 인수
print(y.data)
```

이와 같이 리스트 입출력을 구현함으로써 여러 개의 변수를 입력 받을 수 있게 되었고, 출력 역시 튜플로 바꿔서 여러 개의 변수에 대응할 수 있도록 하였다.

그러나 이는 다소 불편하다. 사용자에게 입력 변수를 리스트로 입력할 것을 요구하며, 반환값이 튜플이어서 이를 정상적으로 처리하기 위해서는 `y = ys[0]`과 같은 절차가 필요하기 때문이다. 따라서 다음 단계에서는 이를 좀더 자연스럽게 고쳐보겠다.